In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import collections as mc
%matplotlib inline
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm

import os
import sys
import math
import random
from math import radians
import copy
import pickle
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from struct import pack
from shapely.geometry import MultiPolygon

#import xml.etree.ElementTree as ET
import xml.etree.cElementTree as cET

#module_path = os.path.abspath(os.path.join('./1_setup'))
#if module_path not in sys.path:
#    sys.path.append(module_path)
from ev_model.utilities import geometry as evgeom
from ev_model import elements

In [13]:
def parse_secretory_cell(item):
    for child in item.getchildren():
        if child.tag == 'id':
            _id = int(child.text)
        elif child.tag == 'p1_x':
            p1_x = float(child.text)
        elif child.tag == 'p1_y':
            p1_y = float(child.text)
        elif child.tag == 'p2_x':
            p2_x = float(child.text)
        elif child.tag == 'p2_y':
            p2_y = float(child.text)
        # midpoint coordinates
        elif child.tag == 'x':
            x = float(child.text)
        elif child.tag == 'y':
            y = float(child.text)
        elif child.tag == 'direction_x':
            direction_x = float(child.text)
        elif child.tag == 'direction_y':
            direction_y = float(child.text)
        elif child.tag == 'direction_x_unit':
            direction_x_unit = float(child.text)
        elif child.tag == 'direction_y_unit':
            direction_y_unit = float(child.text)
        elif child.tag == 'direction_length':
            direction_length = float(child.text)
            
        elif child.tag == 'normal_x':
            normal_x = float(child.text)
        elif child.tag == 'normal_y':
            normal_y = float(child.text)
        elif child.tag == 'unit_normal_x':
            unit_normal_x = float(child.text)
        elif child.tag == 'unit_normal_y':
            unit_normal_y = float(child.text)
        elif child.tag == 'normal_length':
            normal_length = float(child.text)
            
        elif child.tag == 'angle':
            angle = float(child.text)
        elif child.tag == 'time_last_secreted':
            time_last_secreted = float(child.text)

    diam = np.sqrt((p2_x - p1_x)**2 + (p2_y - p1_y)**2)
    return elements.SecretoryCell(_id, p1_x, p1_y, p2_x, p2_y, x, y, diam, False)

def parse_ciliary_cell(item):
    for child in item.getchildren():
        if child.tag == 'id':
            _id = int(child.text)
        elif child.tag == 'p1_x':
            p1_x = float(child.text)
        elif child.tag == 'p1_y':
            p1_y = float(child.text)
        elif child.tag == 'p2_x':
            p2_x = float(child.text)
        elif child.tag == 'p2_y':
            p2_y = float(child.text)
        # midpoint coordinates
        elif child.tag == 'x':
            x = float(child.text)
        elif child.tag == 'y':
            y = float(child.text)
        elif child.tag == 'direction_x':
            direction_x = float(child.text)
        elif child.tag == 'direction_y':
            direction_y = float(child.text)
        
        elif child.tag == 'direction_x_unit':
            direction_x_unit = float(child.text)
        elif child.tag == 'direction_y_unit':
            direction_y_unit = float(child.text)
        elif child.tag == 'direction_length':
            direction_length = float(child.text)
            
        elif child.tag == 'normal_x':
            normal_x = float(child.text)
        elif child.tag == 'normal_y':
            normal_y = float(child.text)
        elif child.tag == 'unit_normal_x':
            unit_normal_x = float(child.text)
        elif child.tag == 'unit_normal_y':
            unit_normal_y = float(child.text)
        elif child.tag == 'normal_length':
            normal_length = float(child.text)
        
        elif child.tag == 'angle':
            angle = float(child.text)
        elif child.tag == 'time_last_secreted':
            time_last_secreted = float(child.text)

    diam = np.sqrt((p2_x - p1_x)**2 + (p2_y - p1_y)**2)
    return elements.CiliaryCell(_id, p1_x, p1_y, p2_x, p2_y, x, y, diam, False)

In [4]:
degrees_of_freedom = 2
water_viscosity = {20: 0.0010016, 21: 0.0009775, 22: 0.0009544, 23: 0.0009321, 24: 0.0009107, 25: 0.00089}

tempC = 20
const_Boltzmann = 1.3806504e-23
# Temperature (Kelvins) 20C + 273.15
const_Temperature_K = tempC + 273.15
const_water_viscosity = water_viscosity[tempC]

def parse_ev(item):
    radius_m = 0
    bm_vx =0
    bm_vy =0
    
    for child in item.getchildren():
        if child.tag == 'id':
            _id = int(child.text)
        elif child.tag == 'x':
            x = float(child.text)
        elif child.tag == 'y':
            y = float(child.text)
        elif child.tag == 'x_1':
            x_1 = float(child.text)
        elif child.tag == 'y_1':
            y_1 = float(child.text)
        elif child.tag == 'vx':
            vx = float(child.text)
        elif child.tag == 'vy':
            vy = float(child.text)
            
        elif child.tag == 'bm_vx':
            bm_vx = float(child.text)
        elif child.tag == 'bm_vy':
            bm_vy = float(child.text)
        
        elif child.tag == 'radius_um':
            radius_um = float(child.text)
        
        elif child.tag == 'diffusion_rate_um':
            diffusion_rate_um = float(child.text)
        elif child.tag == 'diffusion_rate_m':
            diffusion_rate_m = float(child.text)

        elif child.tag == 'velocity_ums':
            velocity_ums = float(child.text)
        elif child.tag == 'velocity_ms':
            velocity_ms = float(child.text)
        
        elif child.tag == 'closest_ev_id':
            closest_ev_id = int(child.text)
        elif child.tag == 'closest_ev_distance':
            closest_ev_distance = float(child.text[:-1])
        elif child.tag == 'closest_secretory_cell_id':
            closest_secretory_cell_id = int(child.text)
        elif child.tag == 'closest_secretory_cell_distance':
            closest_secretory_cell_distance = float(child.text[:-1])
        elif child.tag == 'closest_ciliary_cell_id':
            closest_ciliary_cell_id = int(child.text)
        elif child.tag == 'closest_ciliary_cell_distance':
            closest_ciliary_cell_distance = float(child.text[:-1])
    
    radius_m = radius_um / 1e+6
    
    ev = elements.EV(_id=_id, x_1=x_1, y_1=y_1, vx=vx, vy=vy, radius_um=radius_um, bm_vx=bm_vx, bm_vy=bm_vy)
    ev.diffusion_rate_m = diffusion_rate_m
    ev.diffusion_rate_um = diffusion_rate_um
    ev.velocity_ums = velocity_ums
    ev.velocity_ms = velocity_ms
    ev.x = x
    ev.y = y
    ev.closest_ev_id = closest_ev_id
    ev.closest_ev_distance = closest_ev_distance
    ev.closest_secretory_cell_id = closest_secretory_cell_id
    ev.closest_secretory_cell_distance = closest_secretory_cell_distance
    ev.closest_ciliary_cell_id = closest_ciliary_cell_id
    ev.closest_ciliary_cell_distance = closest_ciliary_cell_distance
    return ev

In [5]:
def point_to_line_segment_distance(x, y, x1, y1, x2, y2):
    A = x - x1
    B = y - y1
    C = x2 - x1
    D = y2 - y1

    dot = A * C + B * D
    len_sq = C * C + D * D
    param = -1
    projecting_on_segment = False
    xx = 0.
    yy = 0.

    if len_sq != 0: # in case of 0 length line
        param = dot / len_sq

    if param < 0: # the closest point is p1, point not projecting on segment
        xx = x1
        yy = y1
    elif param > 1: # the closest point is p2, point not projecting on segment
        xx = x2
        yy = y2
    else:  # interpolate to find the closes point on the line segment
        projecting_on_segment = True
        xx = x1 + param * C
        yy = y1 + param * D

    # dx / dy is the vector from p0 to the closest point on the segment
    dx = x - xx
    dy = y - yy
    d = np.sqrt(dx * dx + dy * dy)
    
    msg = f'point_to_line_segment_distance(x:{x}, y:{y}, x1:{x1}, y1:{y1}, x2:{x2}, y2:{y2})\n' \
        f'  >>>  A:{A} B:{B} C:{C} D:{D} \n' \
        f'  >>>  dot(A*C+B*D):{dot}, len_sq: {len_sq}, param: {param} \n' \
        f'  >>>  d:{d:.6f}, dx:{dx:.6f}, dy:{dy:.6f}, xx:{xx} yy:{yy}'
    
    return [projecting_on_segment, d, xx, yy, msg]

In [6]:
def dotprod(x1, y1, x2, y2):
    return x1*x2 + y1*y2

def vlength(x, y):
    return np.sqrt(x*x + y*y)

def projection(a_x, a_y, b_x,  b_y):
    """
    Gives the length of the projection of vector A in the direction of vector B
    """
    length = vlength(a_x, a_y)
    length_vec = vlength(b_x, b_y)
    if length == 0 or length_vec == 0:
        return 0
    return (a_x * b_x + a_y * b_y) / length_vec;


def project(v1, v2):
    """
    Returns a vector parallel to v2 with the length of the projection of v2 in v1
    project: function(vec) {
        return vec.para(this.projection(vec));
    }
    """
    mag = projection(v1[0], v1[1], v2[0], v2[1])
    return parallel(v2, mag)

def add_scaled(src_x, src_y, vec_x, vec_y, scale):
    """
    Scales vector b by scale and adds it to vector a
    """
    return np.array([src_x + scale * vec_x, src_y + scale * vec_y])

def parallel(vec, magnitude, debug=False):
    """
    Gives a vector parallel to vector vec and of length magnitude
    """
    if debug:
        print(f'producing a parallel vector to vec:{vec}, by magnitude:{magnitude}')
    vector_length = vlength(vec[0], vec[1])
    if debug:
        print(f'vector_length: {vector_length}')
    return vec * (magnitude / vector_length)

In [7]:
def cell_collision_resolution(cell_id, agent, debug=False):
    cell = cells[cell_id]
    if debug:
        print(' > cell_collision_resolution() based on')
    
    # // vectors from ev to endpoints of wall
    ev_p1x = cell.p1[0] - agent.x
    ev_p1y = cell.p1[1] - agent.y
    ev_p2x = cell.p2[0] - agent.x
    ev_p2y = cell.p2[1] - agent.y
    if debug:
        print(f"  >>  ev_p1x: {ev_p1x:+.6f}, ev_p1y: {ev_p1y:+.6f}")
        print(f"  >>  ev_p2x: {ev_p2x:+.6f}, ev_p2y: {ev_p2y:+.6f}")
    
    #// projection of above vectors onto wall vector
    # var proj1 = ballp1.projection(wdir);
    proj1 = projection(ev_p1x, ev_p1y, cell.direction[0], cell.direction[1])
    proj2 = projection(ev_p2x, ev_p2y, cell.direction[0], cell.direction[1])
    if debug:
        print(f"  >>  projection1: {proj1:+.6f}, projection2: {proj2:+.6f}")

    #// Perpendicular distance vector from the object to the wall
    #// var dist = ballp1.addScaled(wdir.unit(), proj1*(-1));
    dist_x = ev_p1x + cell.direction_unit[0] * -proj1
    dist_y = ev_p1y + cell.direction_unit[1] * -proj1
    dist = add_scaled(ev_p1x, ev_p1y, cell.direction_unit[0], cell.direction_unit[1], -proj1)
    dist_length = vlength(dist_x, dist_y)
    
    if debug:
        print(f'  >>  dist_length: {dist_length:+.6f}, dist_x: {dist_x:+.6f}, dist_y: {dist_y:+.6f} | dist: ({dist[0]:+.6f}, {dist[1]:+.6f})')
    
    #// collision detection
    #//var test = ((Math.abs(proj1) < wdir.length()) && (Math.abs(proj2) < wdir.length()));
    
    test = (abs(proj1) < cell.direction_length) and (abs(proj2) < cell.direction_length)
    if debug:
        print(f'  >>  Should we test for a collision? {test}')
        print(f'  >>  Collision straight with the cell? (dist.length() < radius_um): {dist_length < agent.radius_um} && test')
        print(f'  >>  Collision with 1st edge? vlength(ev_p1x, ev_p1y): {vlength(ev_p1x, ev_p1y):+.6f} < radius? {vlength(ev_p1x, ev_p1y) < agent.radius_um}')
        print(f'  >>  Collision with 2nd edge? vlength(ev_p2x, ev_p2y): {vlength(ev_p2x, ev_p2y):+.6f} < radius? {vlength(ev_p2x, ev_p2y) < agent.radius_um}')

In [8]:
def cell_collision_recomputation(future_x, future_y, vx_1, vy_1, agent_radius_um, cell, agent, debug=False):
    """
    Should return:
    int type - type of collision [0:no collision, 1:collision with segment, 2:collision with first end point, 3: collision with second end point]
    float x - x component of the post collision position
    float y - y component of the post collision position
    float vx - x component of the post collision velocity
    float vy - y component of the post collision velocity
    """
    collision_type = 0
    new_x = 0
    new_y =0
    new_vx = 0
    new_vy=0
    # starting velocity
    v_1 = np.array([vx_1, vy_1])
    
    # vectors from ev to endpoints of wall
    ev_p1x = cell.p1[0] - future_x
    ev_p1y = cell.p1[1] - future_y
    ev_p2x = cell.p2[0] - future_x
    ev_p2y = cell.p2[1] - future_y
    if debug:
        print(' >> cell_collision_recomputation()')
        print(f'    >> Initial velocities: {v_1}, wall/cell direction: {cell.direction[0], cell.direction[1]}')
        print(f"  >>>>  ev_p1x: {ev_p1x:+.6f}, ev_p1y: {ev_p1y:+.6f}")
        print(f"  >>>>  ev_p2x: {ev_p2x:+.6f}, ev_p2y: {ev_p2y:+.6f}")
    
    proj1 = projection(ev_p1x, ev_p1y, cell.direction[0], cell.direction[1])
    proj2 = projection(ev_p2x, ev_p2y, cell.direction[0], cell.direction[1])
        
    cell_dir_unit = cell.direction / (cell.direction**2).sum()**0.5
    
    if debug:
        print(f"  >>>>  projection1: {proj1:+.6f}")
        print(f"  >>>>  projection2: {proj2:+.6f}")
        print(f"  >>>>  cell_dir_unit: {cell_dir_unit}")
    
    #// Perpendicular distance vector from the object to the wall
    perp_dist_x = ev_p1x + cell_dir_unit[0] * -proj1
    perp_dist_y = ev_p1y + cell_dir_unit[1] * -proj1
    perp_dist_length = vlength(perp_dist_x, perp_dist_y)
    perp_dist = add_scaled(ev_p1x, ev_p1y, cell_dir_unit[0], cell_dir_unit[1], -proj1)
    
    test = (abs(proj1) < cell.direction_length) and (abs(proj2) < cell.direction_length)
    if debug:
        print(f'  >>>>  perp_dist_length: {perp_dist_length:+.6f}, perp_dist_x: {perp_dist_x:+.6f}, perp_dist_y: {perp_dist_y:+.6f} | perp_dist: ({perp_dist[0]:+.6f}, {perp_dist[1]:+.6f})')
        print(f'  >>>>  Should we test for a collision? {test}')
        
    if perp_dist_length < agent_radius_um and test:
        collision_type = 1
        # var angle = Vector2D.angleBetween(obj.velo2D, wdir);
        
        dot_product = (dotprod(vx_1, vy_1, cell.direction[0], cell.direction[1]) / (vlength(vx_1, vy_1) * cell.direction_length))
        angle = np.arccos(dot_product)
        if debug:
            print(f'  >>>> Solving collision straight with the cell')
            print(f'    >> dot_product: {dot_product:.6f}, angle: {angle}')
        
        normal_dot_evVelocity = dotprod(cell.unit_normal[0], cell.unit_normal[1], vx_1, vy_1)
        normal_x = cell.unit_normal[0]
        normal_y = cell.unit_normal[1]
        if normal_dot_evVelocity > 0:
            normal_x *= -1.0
            normal_y *= -1.0
        
        deltaS = (agent_radius_um + dotprod(perp_dist_x, perp_dist_y, normal_x, normal_y)) / np.sin(angle)
        if debug:
            print(f'    >> normal_dot_evVelocity: {normal_dot_evVelocity}, new normal (x: {normal_x}, y: {normal_y})')
            print(f'    >> deltaS: {deltaS}')
        
        # 4. estimate what was the displacement, so we compute a vector parallel to 'the velocity vector of this timestep' and of length deltaS
        displacement = parallel(v_1, deltaS, debug)

        # 5. update position by subtracting the displacement
        new_x = future_x - displacement[0]
        new_y = future_y - displacement[1]
        if debug:
            print(f'    >> displacement: (x:{displacement[0]}, y:{displacement[1]}) <---- Error here?')
            print(f'    >> new_x: {new_x}, new_y: {new_y}')
        
        # the correction factor was skipped but in the original implementation it is computes as follows
        # var vcor = 1-acc.dotProduct(displ)/obj.velo2D.lengthSquared();
        #new_velocity_x = vx_1  # *vcor;
        #new_velocity_y = vy_1  # *vcor;
        
        # 6. decompose the velocity
        # velocity vector component perpendicular to wall just before impact
        # var normalVelo = dist.para(Velo.projection(dist));
        velocityProjection = projection(vx_1, vy_1, perp_dist_x, perp_dist_y)
        normalVelocity = parallel(perp_dist, velocityProjection, debug)
        if debug:
            print(f'    >> velocityProjection:{velocityProjection}, normalVelocity:{normalVelocity}')
        
        # velocity vector component parallel to wall; unchanged by impact
        # var tangentVelo = Velo.subtract(normalVelo);
        tangentVelocity_x = vx_1 - normalVelocity[0]
        tangentVelocity_y = vy_1 - normalVelocity[1]
        if debug:
            print(f'    >> tangentVelocity_x:{tangentVelocity_x}, tangentVelocity_y:{tangentVelocity_y}')
        
        # velocity vector component perpendicular to wall just after impact
        # obj.velo2D = tangentVelo.addScaled(normalVelo, -vfac);
        new_vx = tangentVelocity_x + (-normalVelocity[0])
        new_vy = tangentVelocity_y + (-normalVelocity[1])
        if debug:
            print(f'    >> new_vx: {new_vx}, new_vy: {new_vy}')
    else:
        if debug:
            print('  >>>>  No direct collision with the wall')
        pass
    if vlength(ev_p1x, ev_p1y) < agent_radius_um:
        collision_type = 2
        
        if debug:
            print(f'  >>>>  Collision with 1st edge? vlength(ev_p1x, ev_p1y): {vlength(ev_p1x, ev_p1y):+.6f} < radius? {vlength(ev_p1x, ev_p1y) < agent_radius_um}')
    else:
        if debug:
            print('  >>>>  No collision with 1st edge')
    if vlength(ev_p2x, ev_p2y) < agent_radius_um:
        collision_type = 3
        if debug:
            print(f'  >>>>  Collision with 2nd edge? vlength(ev_p2x, ev_p2y): {vlength(ev_p2x, ev_p2y):+.6f} < radius? {vlength(ev_p2x, ev_p2y) < agent_radius_um}')
    else:
        if debug:
            print('  >>>>  No collision with 2nd edge')
            
    return [collision_type, new_x, new_y, new_vx, new_vy]

In [71]:
def ev_collision_validation(ev1, ev2, debug=False):
    print('----------\n ev_collision_validation')
    # check the distance between their centers
    min_distance = ev1.radius_um + ev2.radius_um
    dist = ev1.location() - ev2.location()
    dist_length = vlength(dist[0], dist[1])
    print(f'min_distance:{min_distance}, dist:{dist}, dist_length:{dist_length}')
    if dist_length < min_distance:
        print('COLLISION')
        print(f'starting pos ev1 [{ev1.x_1}, {ev1.y_1}], ev2 [{ev2.x_1}, {ev2.y_1}]')
        # // normal velocity vectors just before the impact
        # var normalVelo1 = ball1.velo2D.project(dist);
        # var normalVelo2 = ball2.velo2D.project(dist);
        ev1_velo = np.array([ev1.vx, ev1.vy])
        ev2_velo = np.array([ev2.vx, ev2.vy])
        normal_velocity1 = project(ev1_velo, dist)
        normal_velocity2 = project(ev2_velo, dist)
        print(f'ev1_velo:{ev1_velo}, ev2_velo:{ev2_velo}, normal_velocity1:{normal_velocity1}, normal_velocity2:{normal_velocity2}')
        print(f'ev1_unit:{ ev1.vx / vlength(ev1.vx, ev1.vy)},{ ev1.vy / vlength(ev1.vx, ev1.vy)}, ev2_unit:{ ev2.vx / vlength(ev2.vx, ev2.vy)},{ ev2.vy / vlength(ev2.vx, ev2.vy)}')
        dp = normal_velocity1[0] * normal_velocity2[0] + normal_velocity1[1] * normal_velocity2[1]
        if dp < 0:
            print('Oposite directions')
        elif dp > 0:
            print('SAME DIRECTIONS')
        else:
            print('ORTHOGONAL!!!!!')
        
        # // tangential velocity vectors
        # var tangentVelo1 = ball1.velo2D.subtract(normalVelo1);
        # var tangentVelo2 = ball2.velo2D.subtract(normalVelo2);
        tangent_velocity1 = ev1_velo - normal_velocity1
        tangent_velocity2 = ev2_velo - normal_velocity2
        print(f'tangent_velocity1:{tangent_velocity1}, tangent_velocity2:{tangent_velocity2}')
        
        # // move particles so that they just touch	
        # var L = ball1.radius + ball2.radius-dist.length();
        # var vrel = normalVelo1.subtract(normalVelo2).length();
        # ball1.pos2D = ball1.pos2D.addScaled(normalVelo1,-L/vrel);
        # ball2.pos2D = ball2.pos2D.addScaled(normalVelo2,-L/vrel);
        L = min_distance - dist_length
        normal_velo_subtracted = normal_velocity1 - normal_velocity2
        vrel = vlength(normal_velo_subtracted[0], normal_velo_subtracted[1])
        # (src_x, src_y, vec_x, vec_y, scale):
        ev1_loc = ev1.location()
        ev2_loc = ev2.location()
        if dp > 0:
            v1mag = np.sqrt(ev1.vx*ev1.vx+ev1.vy*ev1.vy)
            v2mag = np.sqrt(ev2.vx*ev2.vx+ev2.vy*ev2.vy)
            print(f'ev1 mass:{ev1.mass_ag} vel:{v1mag}, ev2 mass:{ev2.mass_ag} mag:{v2mag}')
            if ev1.mass_ag > ev2.mass_ag:
                if v1mag > v2mag:
                    print("EV1 heavier and faster")
                    new_pos1 = add_scaled(ev1_loc[0], ev1_loc[1], normal_velocity1[0], normal_velocity1[1], -L/vrel)
                    new_pos2 = add_scaled(ev2_loc[0], ev2_loc[1], normal_velocity2[0], normal_velocity2[1], L/vrel)
                else:
                    print("EV1 heavier but slower")
                    new_pos1 = add_scaled(ev1_loc[0], ev1_loc[1], normal_velocity1[0], normal_velocity1[1], L/vrel)
                    new_pos2 = add_scaled(ev2_loc[0], ev2_loc[1], normal_velocity2[0], normal_velocity2[1], -L/vrel)
            else:
                if v1mag > v2mag:
                    print("EV2 heavier but slower")
                    new_pos1 = add_scaled(ev1_loc[0], ev1_loc[1], normal_velocity1[0], normal_velocity1[1], L/vrel)
                    new_pos2 = add_scaled(ev2_loc[0], ev2_loc[1], normal_velocity2[0], normal_velocity2[1], -L/vrel)
                else:
                    print("EV2 heavier and faster")
                    new_pos1 = add_scaled(ev1_loc[0], ev1_loc[1], normal_velocity1[0], normal_velocity1[1], -L/vrel)
                    new_pos2 = add_scaled(ev2_loc[0], ev2_loc[1], normal_velocity2[0], normal_velocity2[1], L/vrel)
        else:
            new_pos1 = add_scaled(ev1_loc[0], ev1_loc[1], normal_velocity1[0], normal_velocity1[1], -L/vrel)
            new_pos2 = add_scaled(ev2_loc[0], ev2_loc[1], normal_velocity2[0], normal_velocity2[1], -L/vrel)
        new_pos1_old = add_scaled(ev1_loc[0], ev1_loc[1], normal_velocity1[0], normal_velocity1[1], -L/vrel)
        new_pos2_old = add_scaled(ev2_loc[0], ev2_loc[1], normal_velocity2[0], normal_velocity2[1], -L/vrel)
        print(f'normal_velo_subtracted:{normal_velo_subtracted}')
        print(f'L:{L}, vrel:{vrel}, fac:{L/vrel}')
        
        # // normal velocity components after the impact
        # var m1 = ball1.mass;
        # var m2 = ball2.mass;
        # var u1 = normalVelo1.projection(dist);
        # var u2 = normalVelo2.projection(dist);
        # var v1 = ((m1-m2)*u1+2*m2*u2)/(m1+m2);
        # var v2 = ((m2-m1)*u2+2*m1*u1)/(m1+m2);
        u1 = projection(normal_velocity1[0], normal_velocity1[1], dist[0], dist[1])
        u2 = projection(normal_velocity2[0], normal_velocity2[1], dist[0], dist[1])
        #v1 = ((ev1.mass_kg - ev2.mass_kg)*u1+2*ev2.mass_kg*u2) / (ev1.mass_kg + ev2.mass_kg)
        #v2 = ((ev2.mass_kg - ev1.mass_kg)*u2+2*ev1.mass_kg*u1) / (ev1.mass_kg + ev2.mass_kg)
        v1 = ((ev1.mass_ag - ev2.mass_ag)*u1+2*ev2.mass_ag*u2) / (ev1.mass_ag + ev2.mass_ag)
        v2 = ((ev2.mass_ag - ev1.mass_ag)*u2+2*ev1.mass_ag*u1) / (ev1.mass_ag + ev2.mass_ag)
        print(f'u1:{u1}, u2:{u2}, v1:{v1}, v2:{v2}')
        
        # // normal velocity vectors after collision
        # normalVelo1 = dist.para(v1);
        # normalVelo2 = dist.para(v2);
        normal_velocity1 = parallel(dist, v1)
        normal_velocity2 = parallel(dist, v2)
        print(f'normal_velocity1:{normal_velocity1}, normal_velocity2:{normal_velocity2}')
        
        # // final velocity vectors after collision
        # ball1.velo2D = normalVelo1.add(tangentVelo1);
        # ball2.velo2D = normalVelo2.add(tangentVelo2);
        new_vel1 = normal_velocity1 + tangent_velocity1
        new_vel2 = normal_velocity2 + tangent_velocity2
        print(f'new_pos1:{new_pos1}, before_fix:{new_pos1_old}, new_vel1:{new_vel1}, new vlength(new_vel1):{vlength(new_vel1[0], new_vel1[1])}')
        print(f'new_pos2:{new_pos2}, before_fix:{new_pos2_old}, new_vel2:{new_vel2}, new vlength(new_vel2):{vlength(new_vel2[0], new_vel2[1])}')
        print(f'new distance: {np.sqrt((new_pos1[0]-new_pos2[0])**2 + (new_pos1[1]-new_pos2[1])**2)}, old distance: {np.sqrt((new_pos1_old[0]-new_pos2_old[0])**2 + (new_pos1_old[1]-new_pos2_old[1])**2)}')
        print(f'Displacement from starting pos to new pos. With new method: {np.sqrt((new_pos1[0]-ev1.location()[0])**2 + (new_pos1[1] * ev1.location()[1])**2)}, old method:{np.sqrt((new_pos1_old[0]-ev1.location()[0])**2 + (new_pos1_old[1] * ev1.location()[1])**2)}')
        print(f'Displacement from starting pos to new pos. With new method: {np.sqrt((new_pos2[0]-ev2.location()[0])**2 + (new_pos2[1] * ev2.location()[1])**2)}, old method:{np.sqrt((new_pos2_old[0]-ev2.location()[0])**2 + (new_pos2_old[1] * ev2.location()[1])**2)}')
        return [True, new_pos1, new_pos2, new_vel1, new_vel2]
    return [False, 0, 0, 0, 0]

In [22]:
import sys
def ev_collision_old(ev1, ev2):
    print('----------\n ev_collision_old')
    min_distance = ev1.radius_um + ev2.radius_um
    distance = ev1.location() - ev2.location()
    distance_mag = vlength(distance[0], distance[1])
    # // compute the direction resulting of the collision
    cs = (ev2.x - ev1.x) / distance_mag if distance_mag !=0 else sys.float_info.epsilon
    sc = (ev2.y - ev1.y) / distance_mag if distance_mag !=0 else sys.float_info.epsilon

    # // calculate the component of velocity in direction cs, sc for each particle
    vp1 = ev1.vx * cs + ev1.vy * sc
    vp2 = ev2.vx * cs + ev2.vy * sc

    vp1vp2 = vp1 - vp2
    # // back to collision time
    ddt = (min_distance - distance_mag) / vp1vp2 if vp1vp2 != 0 else sys.float_info.epsilon

    # // time of collision occuring after the time step ? don't affect current velocities
    # // otherwise, get the next position back to the moment of collision
    if ddt > dt:
        ddt = 0
    elif ddt < 0:
        ddt = dt

    # agent->x -= message->ev1_vx * ddt; // ev1 - agent->x
    # agent->y -= message->ev1_vy * ddt;
    new_x = ev1.x - ev1.vx * ddt  #; // ev1 - agent->x
    new_y = ev1.y - ev1.vy * ddt

    # // calculate components of velocity
    dx = ev2.x - ev1.x
    dy = ev2.y - ev1.y

    # // where x1, y1 are center of ball1, and x2, y2 are center of ball2
    distance2 = np.sqrt(dx*dx + dy*dy)
    # // Unit vector in the direction of the collision
    ax = dx / (distance2 + sys.float_info.epsilon);
    ay = dy / (distance2 + sys.float_info.epsilon);
    # // Projection of the velocities in these axes
    # // vector sum for velocity p1, p2
    va1 = ev1.vx * ax + ev1.vy * ay
    vb = -ev1.vx * ay + ev1.vy * ax
    va2 = ev2.vx * ax + ev2.vy * ay
    # // New velocities in these axes(after collision)

    masses_plus_one = 1 + ev1.mass_kg / ev2.mass_kg
    vaP = va1 + 2 * (va2 - va1) / masses_plus_one if masses_plus_one != 0 else sys.float_info.epsilon

    # // undo the projections, get back to the original coordinates
    # agent->vx = vaP * ax - vb * ay;
    # agent->vy = vaP * ay + vb * ax;
    new_vx = vaP * ax - vb * ay
    new_vy = vaP * ay + vb * ax #// new vx, vy for ball 1 after collision;
    new_velocity_ums = np.sqrt(new_vx * new_vx + new_vy * new_vy);
    # // Because we have moved back the time by dt, we need to move the time forward by dt.
    # agent->x += agent->vx * (dt - ddt);
    # agent->y += agent->vy * (dt - ddt);
    #new_x += new_vx * (dt - ddt)
    #new_y += new_vy * (dt - ddt)
    print(f'new_x:{new_x}, new_y:{new_y}, new_vx:{new_vx}, new_vy:{new_vy}, new_velocity_ums:{new_velocity_ums}')

In [10]:
def test_direction_change(prev_ev, curr_ev):
    if round(prev_ev.vx, 6) != round(curr_ev.vx, 6) or round(prev_ev.vy, 6) != round(curr_ev.vy, 6):
        # print(f'DirChg-> EV:{curr_ev._id} vx:{prev_ev.vx} -> vx:{curr_ev.vx}, vy:{prev_ev.vy} -> vy:{curr_ev.vy}')
        pass

def test_velocity_change(prev_ev, curr_ev):
    if round(prev_ev.velocity_ums, 6) != round(curr_ev.velocity_ums, 6):
        print(f'EV {curr_ev._id} velocity changed!!! prev: {prev_ev.velocity_ums}, new: {curr_ev.velocity_ums}')
        return True
    return False

def test_collision_with_another_ev(ev):
    if ev.closest_ev_id != -1 or ev.closest_ev_distance < 100:
        print(f'EvEv---> EV {ev._id} collided with ev {ev.closest_ev_id} at distance {ev.closest_ev_distance}')
        return [True, ev.closest_ev_id, ev.closest_ev_distance]
    return [False, -1, 100]   

def test_collision_with_wall(ev):
    if ev.closest_ciliary_cell_id != -1 or ev.closest_ciliary_cell_distance < 100 or ev.closest_secretory_cell_id != -1 or ev.closest_secretory_cell_distance < 100:
        c_cc_id = iterations[i][ev_id].closest_ciliary_cell_id
        c_cc_d = iterations[i][ev_id].closest_ciliary_cell_distance
        c_sc_id = iterations[i][ev_id].closest_secretory_cell_id
        c_sc_d = iterations[i][ev_id].closest_secretory_cell_distance
        print(f'EvWll--> EV:[id:{ev._id}, x:{ev.x}, y:{ev.y}, r:{ev.radius_um}, vel_ums:{ev.velocity_ums}]', end=' ')
        return [True, c_cc_id, c_cc_d, c_sc_id, c_sc_d]
    return [False, -1, 100, -1, 100]

def print_wall_collision_info(c_cc_id, c_cc_d, c_sc_id, c_sc_d):
    print('|', end=' ')
    if c_cc_id > -1:
        print(f'{cells[c_cc_id].name}:[id:{c_cc_id}, d:{c_cc_d}, p1:({cells[c_cc_id].p1[0]}, {cells[c_cc_id].p1[1]}), p2:({cells[c_cc_id].p2[0]}, {cells[c_cc_id].p2[1]})]', end='')
    if c_cc_id > -1 and c_sc_id > -1:
        print(', ', end='')
    if c_sc_id > -1:
        print(f'{cells[c_sc_id].name}:[id:{c_sc_id}, d:{c_sc_d}, p1:({cells[c_sc_id].p1[0]}, {cells[c_sc_id].p1[1]}), p2:({cells[c_sc_id].p2[0]}, {cells[c_sc_id].p2[1]})]', end='')
    print('')

def different_rounded_values(v1, v2, precision=4):
    return round(v1, precision) != round(v2, precision)

In [70]:
print(iterations[3][27].get_xml_description(2, 0))
print(iterations[3][35].get_xml_description(2, 0))


    <xagent>
        <name>EV</name>
        <id>27</id>
        <x>8.115994</x>
        <y>3.550756</y>
        <x_1>8.121110</x_1>
        <y_1>3.545640</y_1>
        <vx>0.1211750000</vx>
        <vy>-0.1211750000</vy>
        <bm_vx>0</bm_vx>
        <bm_vy>0</bm_vy>
        <mass_kg>1.6325190966456338612868785818677168e-12f</mass_kg>
        <mass_ag>1632.519097f</mass_ag>
        <colour/>
        <radius_um>0.155000</radius_um>
        <radius_m>0.0000001550</radius_m>
        <diffusion_rate_um>1.3830780000000000296012104</diffusion_rate_um>
        <diffusion_rate_m>0.0000000000000000000000000</diffusion_rate_m>
        <diff_rate_um_x_twice_dof>5.5323120000000001184048415</diff_rate_um_x_twice_dof>
        <velocity_ums>0.1663180000</velocity_ums>
        <velocity_ms>0.0000000000</velocity_ms>
        <closest_ev_id>-1</closest_ev_id>
        <closest_ev_distance>100</closest_ev_distance>
        <closest_secretory_cell_id>-1</closest_secretory_cell_id>
        <closest_sec

# Load the iterations data

In [51]:
base_path = 'D:\\iterations\\v28\\vfast\\'
zero_xml = '0_0.0k_intCol_newEVs.xml'
files = [zero_xml if j==0 else f'{j}.xml' for j in range(100)]

iterations = {}
evs_zero = {}
cells = {}

for i in range(len(files)):
    evs = {}
    cur_reading = os.path.join(base_path, files[i])
    if i%250==0:
        print(f'Curently reading {cur_reading}')
    with open(cur_reading, 'r') as xml_in:
        tree = cET.parse(xml_in)
        root = tree.getroot()
        if i == 0:
            dt = float(root.find('./environment/dt').text)

        for item in root.findall('./xagent'):
            cell = None
            ev = None
            for child in item:
                if child.tag == 'name':
                    if child.text == 'SecretoryCell' and i == 0:
                        cell = parse_secretory_cell(item)
                        cells[cell._id] = cell
                    elif child.text == 'CiliaryCell' and i == 0:
                        cell = parse_ciliary_cell(item)
                        cells[cell._id] = cell
                    elif child.text == 'EV':
                        ev = parse_ev(item)
                        if i == 0:
                            evs_zero[ev._id] = ev
                        else:
                            # check the mass
                            if ev.mass_kg != evs_zero[ev._id].mass_kg:
                                print(f'mass matches? {ev.mass_kg == evs_zero[_id].mass_kg} ev.mass_Kg: {ev.mass_kg}, mass_kg: {evs_zero[_id].mass_kg}')
                        #ev.radius_um = evs_zero[ev._id].radius_um
                        evs[ev._id] = ev
                        
    iterations[i] = evs
len(iterations)

Curently reading D:\iterations\v28\vfast\0_0.0k_intCol_newEVs.xml


100

In [72]:
debug = False
for i in range(1, 10):  #len(iterations)):
    #if i>560 and i < 570 or i > 750 and i < 760 or i > 790 and i < 810 or i > 975 and i < 990:
    #    print(f't{i}')
    #else:
    print(f'------\nt{i}')

    for ev_id in [k for k in iterations[i].keys()]:
        ev = iterations[i][ev_id]
        ev_prev = iterations[i-1][ev_id]
        print(f'EV:{ev_id}')
        test_direction_change(iterations[i-1][ev_id], ev)
        
        if test_velocity_change(iterations[i-1][ev_id], ev):
            print('velocity changed incorrectly @T:',i)
        
        [collided_with_ev, cl_ev_id, cl_ev_d] = test_collision_with_another_ev(ev)
        if collided_with_ev:
            # double check evev collision resolution
            ev2 = iterations[i][cl_ev_id]
            ev_collision_validation(ev, ev2)
            #ev_collision_old(ev, ev2)
        else:
            print('No collision in simulation')
        
        [collided_with_wall, c_cc_id, c_cc_d, c_sc_id, c_sc_d] = test_collision_with_wall(ev)
        if collided_with_wall:
            print_wall_collision_info(c_cc_id, c_cc_d, c_sc_id, c_sc_d)
            # if the cell collided with a wall we double check here if the collision was possible 
            # based on the displacement from the previous position
            

            # following checks are based on the EVs last position, which was corrected if a collision was detected
            clc = None
            if c_cc_id > -1:
                clc = cells[c_cc_id]
                
                [projecting_on_segment, d, xx, yy, _] = point_to_line_segment_distance(ev.x, ev.y, clc.p1[0], clc.p1[1], clc.p2[0], clc.p2[1])
                if debug:
                    print(f'>>> d:({d}) < radius_um?: {d < ev.radius_um}, projecting_on_segment? {projecting_on_segment}')
                #cell_collision_resolution(c_cc_id, ev, debug)
                
            if c_sc_id > -1:
                clc = cells[c_sc_id]
                
                [projecting_on_segment, d, xx, yy, _] = point_to_line_segment_distance(ev.x, ev.y, clc.p1[0], clc.p1[1], clc.p2[0], clc.p2[1])
                if debug:
                    print(f'>>> d:({d}) < radius_um?: {d < ev.radius_um}, projecting_on_segment: {projecting_on_segment}')
                #cell_collision_resolution(c_sc_id, ev, debug)

            
            if debug:
                print('double checking the collision detection in python:')
                print(f'Brownian motion? {ev.bm_vx}, {ev.bm_vy}')
            # check if the EV was affected by brownian motion
            #if (ev.bm_vx != 0 and ev_prev.vx != ev.bm_vx) or (ev.bm_vy != 0 and ev_prev.vy != ev.bm_vy):
            if ev.bm_vx != 0 or ev.bm_vy !=0:
                future_x = ev.x_1 + (ev.bm_vx * dt)
                future_y = ev.y_1 + (ev.bm_vy * dt)
                vx_1 = ev.bm_vx
                vy_1 = ev.bm_vy
                if debug:
                    print(f'Initial position: ({ev.x_1}, {ev.y_1}), future position before collision resolution:({future_x}, {future_y}), BM: ({ev.bm_vx}, {ev.bm_vy})')
            else:
                future_x = ev.x_1 + (ev_prev.vx * dt)
                future_y = ev.y_1 + (ev_prev.vy * dt)
                vx_1 = ev_prev.vx
                vy_1 = ev_prev.vy
                if debug:
                    print(f'Initial position: ({ev.x_1}, {ev.y_1}), future position before collision resolution:({future_x}, {future_y})')
                    
            [projecting_on_segment, d, xx, yy, msg] = point_to_line_segment_distance(future_x, future_y, clc.p1[0], clc.p1[1], clc.p2[0], clc.p2[1])
            if debug:
                print(f'>>> d:{d:.6f} < radius_um?: {d < ev.radius_um}, projecting_on_segment? {projecting_on_segment} closest point on segment ({xx}, {yy})')
                #print('---- ', end='')
                #print(msg)
                #print('----')
            [collision_type, new_x, new_y, new_vx, new_vy] = cell_collision_recomputation(future_x, future_y, vx_1, vy_1, ev.radius_um, clc, ev, debug)
            # we must check these values against those in the XML file
            if collision_type == 0:
                raise RuntimeError('No collision should have happened with ', clc._id)
            if different_rounded_values(ev.x, new_x) or different_rounded_values(ev.y, new_y) or different_rounded_values(ev.vx, new_vx) or different_rounded_values(ev.vy, new_vy):
                if different_rounded_values(ev.x, new_x):
                    print(f'ev.x:{ev.x}, new_x:{new_x}')
                if different_rounded_values(ev.y, new_y):
                    print(f'ev.y:{round(ev.y, 4)}, new_y:{round(new_y, 5)}')
                if different_rounded_values(ev.vx, new_vx):
                    print(f'ev.vx:{ev.vx}, new_vx:{new_vx}')
                if different_rounded_values(ev.vy, new_vy):
                    print(f'ev.vy:{ev.vy}, new_vy:{new_vy}')
                raise RuntimeError('The values post collision resolution does not match python <> cuda!! cell_id:',clc._id,' @T:',i)
            else:
                print(' >> Collision resolved OK and validated in Python')

------
t1
EV:27
No collision in simulation
EV:35
No collision in simulation
------
t2
EV:27
No collision in simulation
EV:35
No collision in simulation
------
t3
EV:27
EvEv---> EV 27 collided with ev 35 at distance 0.301
----------
 ev_collision_validation
min_distance:0.301, dist:[ 0.212832 -0.212845], dist_length:0.3009990934355124
COLLISION
starting pos ev1 [8.12111, 3.54564], ev2 [7.908277, 3.758486]
ev1_velo:[ 0.121175 -0.121175], ev2_velo:[ 0.121172 -0.121172], normal_velocity1:[ 0.1211713 -0.1211787], normal_velocity2:[ 0.1211683 -0.1211757]
ev1_unit:0.7071067811865476,-0.7071067811865476, ev2_unit:0.7071067811865476,-0.7071067811865476
SAME DIRECTIONS
tangent_velocity1:[3.70074753e-06 3.70052150e-06], tangent_velocity2:[3.70065591e-06 3.70042988e-06]
ev1 mass:1632.5190966456337 vel:0.1713673284205598, ev2 mass:1364.3372700099642 mag:0.17136308577987266
EV1 heavier and faster
normal_velo_subtracted:[ 2.99990838e-06 -3.00009162e-06]
L:9.065644875905221e-07, vrel:4.242640685127715